In [1]:
import pandas as pd
import numpy as np

In [2]:
from pathlib import Path

In [3]:
dPath = Path("../docs/dumps")

In [4]:
import pickle

In [5]:
with open(dPath / "train_data.pkl", 'rb') as filename:
    train_data = pickle.load(filename)

In [6]:
with open(dPath / "valid_data.pkl", 'rb') as filename:
    valid_data = pickle.load(filename)

In [7]:
X_train = train_data.drop("Detected", axis=1)

In [8]:
y_train = train_data.Detected

In [9]:
X_valid = valid_data.drop("Detected", axis=1)

In [10]:
y_valid = valid_data.Detected

In [11]:
with open(dPath / "rf_exp_05_names.pkl", 'rb') as filename:
    names = pickle.load(filename)

In [12]:
X_train = X_train[names] 

In [13]:
X_valid = X_valid[names]

In [14]:
X_train.head()

,numExecuted,MutatorClass,numAssertInTC,numTestCover,numAssertInTM,ppavcc,cchalsteadCumulativeBugs,cchalsteadCumulativeVolume,ppRVF,ppnumberOfMethods,...,cchalsteadEffort,ccmaintainabilityIndexNC,mmhalsteadVocabulary,pptcc,mmexternalMethodsCalled,ppinstability,ppmaxcc,mmhalsteadEffort,mmvariablesReferenced,ccunweightedClassSize
0,14,2,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
1,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
2,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
3,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
4,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0


In [15]:
from imblearn.over_sampling import ADASYN

In [16]:
sm = ADASYN(random_state=42, n_jobs=-1,  n_neighbors=5)

In [17]:
%time X_res, y_res = sm.fit_resample(X_train, y_train)

Wall time: 52.2 s


In [18]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
rf = RandomForestClassifier(n_estimators=100, max_features=0.7, oob_score=True, random_state=42, n_jobs=-1)

In [44]:
%time rf.fit(X_res, y_res)

Wall time: 11min 29s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=0.7,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [45]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [46]:
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid), roc_auc_score(y_valid, m.predict(X_valid))]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [47]:
def conf_matr(m):
    y_pred = m.predict(X_valid)
    print(classification_report(y_valid, y_pred))

In [48]:
print_score(rf)

[0.9084424598858571, 0.6631652246059025, 0.6157093805648253, 0.8601143331422239]


In [49]:
conf_matr(rf)

              precision    recall  f1-score   support

       False       0.37      0.52      0.43     16657
        True       0.82      0.71      0.76     50839

    accuracy                           0.66     67496
   macro avg       0.59      0.62      0.60     67496
weighted avg       0.71      0.66      0.68     67496



In [50]:
with open(dPath / "rf_exp_06.pkl", 'wb') as filename:
    pickle.dump(rf,filename)

In [51]:
names = X_train.columns

In [52]:
feat_importances = pd.Series(rf.feature_importances_, index=names)
feat_importances.sort_values(ascending=False,inplace=True)

In [53]:
feat_importances

numExecuted                      0.153462
MutatorClass                     0.101407
numAssertInTM                    0.050169
numAssertInTC                    0.047850
numTestCover                     0.043426
ppavcc                           0.033671
mmhalsteadDifficulty             0.030779
mmnumberOfExpressions            0.029173
mmhalsteadVocabulary             0.026104
mmexternalMethodsCalled          0.025695
mminstanceVariablesReferenced    0.025504
mmhalsteadEffort                 0.025440
mmvariablesReferenced            0.023069
cchalsteadCumulativeVolume       0.021664
ccmaintainabilityIndex           0.021326
cchalsteadCumulativeBugs         0.019617
returnType                       0.019557
ccmaintainabilityIndexNC         0.018656
ppmaintainabilityIndex           0.018500
pptcc                            0.017492
ppmaintainabilityIndexNC         0.016733
ccimportedPackages               0.016315
ccunweightedClassSize            0.015437
ccexternalMethodCalls            0

In [54]:
with open(dPath / "rf_exp_06_names.pkl", 'wb') as filename:
    pickle.dump(names,filename)